In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/64 x 64 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_64x64_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'DCT_64_Depth5/5 Projections'

DCT_10_proj5_weights, DCT_10_proj5_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj5', display=True)

DCT_20_proj5_weights, DCT_20_proj5_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj5', display=True)

DCT_30_proj5_weights, DCT_30_proj5_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj5', display=True)

DCT_40_proj5_weights, DCT_40_proj5_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, DCT_10_proj5_weights, DCT_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, DCT_20_proj5_weights, DCT_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, DCT_30_proj5_weights, DCT_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, DCT_40_proj5_weights, DCT_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.05655643938150671
Average test loss: 0.00712753005946676
Epoch 2/300
Average training loss: 0.023064448446035384
Average test loss: 0.0043023791377329165
Epoch 3/300
Average training loss: 0.021623671904206276
Average test loss: 0.00416704971405367
Epoch 4/300
Average training loss: 0.02111254774199592
Average test loss: 0.004105091603265869
Epoch 5/300
Average training loss: 0.020835365363293223
Average test loss: 0.004063301986911231
Epoch 6/300
Average training loss: 0.02062974299987157
Average test loss: 0.00403718104130692
Epoch 7/300
Average training loss: 0.020465954151418473
Average test loss: 0.004006370697998338
Epoch 8/300
Average training loss: 0.020330383674965966
Average test loss: 0.003981315208805932
Epoch 9/300
Average training loss: 0.02021778308517403
Average test loss: 0.003985573992754022
Epoch 10/300
Average training loss: 0.02012985150350465
Average test loss: 0.003955222529255681
Epoch

In [8]:
DCT_10_proj5_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj5_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj5_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj5_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj5_psnr = average_PSNR(DCT_10_proj5_model, DCT_10_proj5_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj5_psnr = average_PSNR(DCT_20_proj5_model, DCT_20_proj5_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj5_psnr = average_PSNR(DCT_30_proj5_model, DCT_30_proj5_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj5_psnr = average_PSNR(DCT_40_proj5_model, DCT_40_proj5_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.89
Average PSNR for Projection Layer 1 across 2500 images: 28.10
Average PSNR for Projection Layer 2 across 2500 images: 28.34
Average PSNR for Projection Layer 3 across 2500 images: 28.45
Average PSNR for Projection Layer 4 across 2500 images: 28.53
Average PSNR for Projection Layer 0 across 2500 images: 28.58
Average PSNR for Projection Layer 1 across 2500 images: 29.34
Average PSNR for Projection Layer 2 across 2500 images: 29.89
Average PSNR for Projection Layer 3 across 2500 images: 30.21
Average PSNR for Projection Layer 4 across 2500 images: 30.43
Average PSNR for Projection Layer 0 across 2500 images: 29.34
Average PSNR for Projection Layer 1 across 2500 images: 30.46
Average PSNR for Projection Layer 2 across 2500 images: 31.14
Average PSNR for Projection Layer 3 across 2500 images: 31.62
Average PSNR for Projection Layer 4 across 2500 images: 31.97
Average PSNR for Projection Layer 0 across 2500 images: 30.04
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'DCT_64_Depth5/15 Projections'

DCT_10_proj15_weights, DCT_10_proj15_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj15', display=True)

DCT_20_proj15_weights, DCT_20_proj15_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj15', display=True)

DCT_30_proj15_weights, DCT_30_proj15_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj15', display=True)

DCT_40_proj15_weights, DCT_40_proj15_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, DCT_10_proj15_weights, DCT_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, DCT_20_proj15_weights, DCT_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, DCT_30_proj15_weights, DCT_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, DCT_40_proj15_weights, DCT_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.8947471406194899
Average test loss: 0.005145649433549907
Epoch 2/300
Average training loss: 0.18164061706595952
Average test loss: 0.004501519038238459
Epoch 3/300
Average training loss: 0.12133590792947345
Average test loss: 0.0043025948032736775
Epoch 4/300
Average training loss: 0.0992928966416253
Average test loss: 0.004211789356958535
Epoch 5/300
Average training loss: 0.08696916284163793
Average test loss: 0.00415490544297629
Epoch 6/300
Average training loss: 0.07946715786059698
Average test loss: 0.004118487681779597
Epoch 7/300
Average training loss: 0.0745587374832895
Average test loss: 0.004085693310118384
Epoch 8/300
Average training loss: 0.07116308510303497
Average test loss: 0.00404561313158936
Epoch 9/300
Average training loss: 0.06867491979731453
Average test loss: 0.004014316964894533
Epoch 10/300
Average training loss: 0.06679546452230878
Average test loss: 0.0039817216195580036
Epoch 11/30

In [10]:
DCT_10_proj15_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj15_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj15_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj15_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj15_psnr = average_PSNR(DCT_10_proj15_model, DCT_10_proj15_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj15_psnr = average_PSNR(DCT_20_proj15_model, DCT_20_proj15_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj15_psnr = average_PSNR(DCT_30_proj15_model, DCT_30_proj15_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj15_psnr = average_PSNR(DCT_40_proj15_model, DCT_40_proj15_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.68
Average PSNR for Projection Layer 1 across 2500 images: 27.92
Average PSNR for Projection Layer 2 across 2500 images: 28.09
Average PSNR for Projection Layer 3 across 2500 images: 28.22
Average PSNR for Projection Layer 4 across 2500 images: 28.23
Average PSNR for Projection Layer 5 across 2500 images: 28.35
Average PSNR for Projection Layer 6 across 2500 images: 28.38
Average PSNR for Projection Layer 7 across 2500 images: 28.38
Average PSNR for Projection Layer 8 across 2500 images: 28.47
Average PSNR for Projection Layer 9 across 2500 images: 28.51
Average PSNR for Projection Layer 10 across 2500 images: 28.52
Average PSNR for Projection Layer 11 across 2500 images: 28.55
Average PSNR for Projection Layer 12 across 2500 images: 28.59
Average PSNR for Projection Layer 13 across 2500 images: 28.61
Average PSNR for Projection Layer 14 across 2500 images: 28.62
Average PSNR for Projection Layer 0 across 2500 images: 28.27
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'DCT_64_Depth5/30 Projections'

DCT_10_proj30_weights, DCT_10_proj30_hist = train_main(DCT_10_normalized, [1] * num_projections, DCT_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_10_proj30', display=True)

DCT_20_proj30_weights, DCT_20_proj30_hist = train_main(DCT_20_normalized, [1] * num_projections, DCT_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_20_proj30', display=True)

DCT_30_proj30_weights, DCT_30_proj30_hist = train_main(DCT_30_normalized, [1] * num_projections, DCT_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_30_proj30', display=True)

DCT_40_proj30_weights, DCT_40_proj30_hist = train_main(DCT_40_normalized, [1] * num_projections, DCT_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/DCT_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, DCT_10_proj30_weights, DCT_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, DCT_20_proj30_weights, DCT_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, DCT_30_proj30_weights, DCT_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, DCT_40_proj30_weights, DCT_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 11.078387171851265
Average test loss: 0.0052490746279557545
Epoch 2/300
Average training loss: 4.137498062557644
Average test loss: 0.2496575379371643
Epoch 3/300
Average training loss: 3.024800004747179
Average test loss: 0.0044553301942845186
Epoch 4/300
Average training loss: 2.346044056150648
Average test loss: 0.004626139213641484
Epoch 5/300
Average training loss: 1.9232318012449476
Average test loss: 0.0042338348047600855
Epoch 6/300
Average training loss: 1.5976563958062067
Average test loss: 0.004172481550110711
Epoch 7/300
Average training loss: 1.3425885925292969
Average test loss: 0.004205736429740985
Epoch 8/300
Average training loss: 1.1375107164382934
Average test loss: 0.0041070586445017
Epoch 9/300
Average training loss: 0.9693000681665208
Average test loss: 0.004093284447780914
Epoch 10/300
Average training loss: 0.83380724785063
Average test loss: 0.004090668128803372
Epoch 11/300
Average tra

In [12]:
DCT_10_proj30_model = MemoryNetwork(DCT_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_20_proj30_model = MemoryNetwork(DCT_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_30_proj30_model = MemoryNetwork(DCT_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
DCT_40_proj30_model = MemoryNetwork(DCT_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

DCT_10_proj30_psnr = average_PSNR(DCT_10_proj30_model, DCT_10_proj30_weights, num_projections, DCT_10_test, print_psnr=True)
DCT_20_proj30_psnr = average_PSNR(DCT_20_proj30_model, DCT_20_proj30_weights, num_projections, DCT_20_test, print_psnr=True)
DCT_30_proj30_psnr = average_PSNR(DCT_30_proj30_model, DCT_30_proj30_weights, num_projections, DCT_30_test, print_psnr=True)
DCT_40_proj30_psnr = average_PSNR(DCT_40_proj30_model, DCT_40_proj30_weights, num_projections, DCT_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(DCT_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 27.54
Average PSNR for Projection Layer 1 across 2500 images: 27.67
Average PSNR for Projection Layer 2 across 2500 images: 27.87
Average PSNR for Projection Layer 3 across 2500 images: 27.96
Average PSNR for Projection Layer 4 across 2500 images: 27.99
Average PSNR for Projection Layer 5 across 2500 images: 28.15
Average PSNR for Projection Layer 6 across 2500 images: 28.15
Average PSNR for Projection Layer 7 across 2500 images: 28.12
Average PSNR for Projection Layer 8 across 2500 images: 28.30
Average PSNR for Projection Layer 9 across 2500 images: 28.27
Average PSNR for Projection Layer 10 across 2500 images: 28.34
Average PSNR for Projection Layer 11 across 2500 images: 28.49
Average PSNR for Projection Layer 12 across 2500 images: 28.37
Average PSNR for Projection Layer 13 across 2500 images: 28.42
Average PSNR for Projection Layer 14 across 2500 images: 28.39
Average PSNR for Projection Layer 15 across 2500 images: 28.39
Av